# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.7.1 サポートベクターマシン (Support Vector Machine, SVM)

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

### あやめのデータセット (iris_with_species.csv)
有名な [Fisher’s Iris Data](https://en.wikipedia.org/wiki/Iris_flower_data_set)。150個のあやめについて、がく片長(Sepal Length)、がく片幅(Sepal Width)、花びら長(Petal Length)、花びら幅(Petal Width)が計測されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0, header=0) # あやめのデータセットの読み込み

SVM は 2 つのクラスを分類するための手法です。あやめのデータセットには、setosa, versicolor, virginica の 3 つのクラスがあるため、setosa と versicolor でまとめて 1 つのクラス `setosa+versicolor` とします。

In [ ]:
dataset.iloc[0:100,0] = 'setosa+versicolor'

In [ ]:
dataset # 念のため確認

In [ ]:
# y と x に分割
y = dataset.iloc[:,0]
x = dataset.iloc[:,1:]

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=50, stratify=y, shuffle=True, random_state=3)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの特徴量の標準化。平均を引いてから、標準偏差で割ります
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの特徴量の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

サポートベクターマシン （Support Vector Machine, SVM）の実行

ガウシアンカーネルを用いた SVM

In [ ]:
from sklearn.svm import SVC # SVM の実行に使用

In [ ]:
model = SVC(kernel='rbf', C=1, gamma=1)

In [ ]:
model.fit(autoscaled_x_train, y_train) # SVMモデル構築

サポートベクターとその係数 (*αy*)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics # 混同行列の作成、正解率の計算に使用

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

クロスバリデーションによる *C* と *γ* の最適化

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
np.arange(-10, 11, 1)

In [ ]:
np.arange(-10, 11, 1.0)

In [ ]:
nonlinear_svm_cs = 2 ** np.arange(-10, 11, 1.0) # Cの候補

In [ ]:
nonlinear_svm_cs # 念のため確認

In [ ]:
nonlinear_svm_gammas = 2 ** np.arange(-20, 11, 1.0) #ガウシアンカーネルのγの候補

In [ ]:
nonlinear_svm_gammas # 念のため確認

クロスバリデーションの設定

In [ ]:
fold_number = 10 # クロスバリデーションのfold数

In [ ]:
from sklearn.model_selection import StratifiedKFold # クロスバリデーションの分割の設定に使用

In [ ]:
fold = StratifiedKFold(n_splits=fold_number, shuffle=True, random_state=9) # クロスバリデーションの分割の設定。(KFold ではなく) StratifiedKFold を使用することで、fold ごとのクラスの割合がなるべく同じになるように分割されます

In [ ]:
model_for_cross_validation = SVC(kernel='rbf')

グリッドサーチの設定

In [ ]:
from sklearn.model_selection import GridSearchCV # グリッドサーチに使用

In [ ]:
gs_cv = GridSearchCV(model_for_cross_validation, {'C':nonlinear_svm_cs, 'gamma':nonlinear_svm_gammas}, cv=fold) # クロスバリデーションによるグリッドサーチの設定

クロスバリデーションによるグリッドサーチの実行

In [ ]:
gs_cv.fit(autoscaled_x_train, y_train)

In [ ]:
optimal_nonlinear_svm_c = gs_cv.best_params_['C'] # 最適な C
optimal_nonlinear_svm_gamma = gs_cv.best_params_['gamma'] # 最適な γ

In [ ]:
optimal_nonlinear_svm_c # 念のため確認

In [ ]:
optimal_nonlinear_svm_gamma # 念のため確認

SVM モデルの構築および予測

In [ ]:
model = SVC(kernel='rbf', C=optimal_nonlinear_svm_c, gamma=optimal_nonlinear_svm_gamma)

In [ ]:
model.fit(autoscaled_x_train, y_train) # SVMモデル構築

サポートベクターとその係数 (*αy*)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train_nlsvm.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train_nlsvm.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

線形カーネルを用いた SVM

In [ ]:
from sklearn.svm import SVC # SVM の実行に使用

In [ ]:
model = SVC(kernel='linear', C=1)

In [ ]:
model.fit(autoscaled_x_train, y_train) # SVMモデル構築

サポートベクターとその係数 (*αy*)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics # 混同行列の作成、正解率の計算に使用

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

クロスバリデーションによる C の最適化

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
linear_svm_cs = 2 ** np.arange(-10, 11, dtype=float) # Cの候補

In [ ]:
linear_svm_cs # 念のため確認

In [ ]:
fold_number = 10 # クロスバリデーションのfold数

In [ ]:
from sklearn.model_selection import StratifiedKFold # クロスバリデーションの設定に使用

In [ ]:
fold = StratifiedKFold(n_splits=fold_number, shuffle=True, random_state=9) # クロスバリデーションの分割の設定。(KFold ではなく) StratifiedKFold を使用することで、fold ごとのクラスの割合がなるべく同じになるように分割されます

In [ ]:
model_for_cross_validation = SVC(kernel='linear')

In [ ]:
from sklearn.model_selection import GridSearchCV # グリッドサーチに使用

In [ ]:
gs_cv = GridSearchCV(model_for_cross_validation, {'C':linear_svm_cs}, cv=fold) # クロスバリデーションによるグリッドサーチの設定

In [ ]:
gs_cv.fit(autoscaled_x_train, y_train) # クロスバリデーションによるグリッドサーチ実行

In [ ]:
optimal_linear_svm_c = gs_cv.best_params_['C'] # 最適な C

In [ ]:
optimal_linear_svm_c # 念のため確認

SVM モデルの構築および予測

In [ ]:
model = SVC(kernel='linear', C=optimal_linear_svm_c)

In [ ]:
model.fit(autoscaled_x_train, y_train) # SVMモデル構築

サポートベクターとその係数 (*αy*)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train_nlsvm.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train_nlsvm.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率